## Librerías Utilizadas

In [ ]:
class HatoDataCleaner:
    def __init__(self, df: pd.DataFrame):
        """
        Inicializa la clase con un DataFrame.
        Se trabaja sobre una copia para no modificar el original.
        """
        self.df = df.copy()

    def filter_groups(self):
        """
        Elimina las categorías que no son relevantes para el estudio
        (ESTABLO INICIAL, MACHOS, etc.).
        """
        groups_to_remove = ['ESTABLO INICIAL', 'MACHOS', 'BECERRAS/NOVILLAS', 'SECAS', 'RETO']
        if 'Nombre del grupo' in self.df.columns:
            self.df = self.df[~self.df['Nombre del grupo'].isin(groups_to_remove)]
        return self

    def select_columns(self):
        """
        Selecciona únicamente las columnas relevantes para el modelo.
        """
        selected_columns = [
            'Número del animal',
            'Estado de la reproducción',
            'Nº Lactación',
            'Días en ordeño',
            'Número de inseminaciones',
            'Días preñada',
            'Días para el parto',
            'Producción de leche ayer',
            'Producción media diaria últimos 7 días',
            'Producción TOTAL en lactación'
        ]
        # Verificar que las columnas existan para evitar errores
        existing_cols = [col for col in selected_columns if col in self.df.columns]
        self.df = self.df[existing_cols]
        return self

    def handle_missing_values(self):
        """
        Sustituye los valores nulos por 0.
        """
        self.df.fillna(0, inplace=True)
        return self

    def run_pipeline(self) -> pd.DataFrame:
        """
        Ejecuta todos los pasos del pipeline secuencialmente y retorna el DataFrame limpio.
        """
        (self.filter_groups()
             .select_columns()
             .handle_missing_values())
        return self.df

In [ ]:
# Ejemplo de uso con los datos originales cargados previamente

# Recargamos los datos crudos para probar el pipeline desde cero
raw_df = pd.read_csv(global_path)

# Instanciamos la clase y corremos el pipeline
cleaner = HatoDataCleaner(raw_df)
df_cleaned_class = cleaner.run_pipeline()

print("Dimensiones del dataset limpio:", df_cleaned_class.shape)
display(df_cleaned_class.head())

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

## sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

## Google Drive
from google.colab import drive

## Sistema Operativo
import os
from typing import List, Union, Optional
from datetime import datetime

## Importación de los datos

Los datos importados solo pueden ser accesados por miembros del equipo Vacas Saturno Saturnitas, por lo que su consulta se realiza exclusivamente a través de carpetas protegidas en Google Drive.

Para obtener acceso, es necesario contactar a los miembros del equipo.

In [ ]:
drive.mount('/content/drive')
global_path = '/content/drive/MyDrive/Vacas/global_hato.csv'
global_hato_df = pd.read_csv(global_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


El dataset de Global Hato contiene informacion global sobre el estado de los animales dentro del CAETEC (vacas, becerros, machos, novillas). Dentro del dataset hay información sobre la producción de leche y el estado reproductivo de la vaca.

In [ ]:
display(global_hato_df.head())

,Número del animal,Nombre del grupo,Número(s) de selección de animal,Estado de la reproducción,Días en ordeño,Nº Lactación,Número de inseminaciones,Días preñada,Días para el parto,Fecha de parto esperada,Producción de leche ayer,Producción media diaria últimos 7 días,Producción TOTAL en lactación
0,5704,VMS 1,1,Inseminada,97.0,7.0,1.0,NaN,NaN,NaN,33.01,31.10,2675.94
1,5921,VMS 1,1,Abierta,65.0,6.0,NaN,NaN,NaN,NaN,33.70,37.79,2606.48
2,6003,VMS 1,1,Inseminada,219.0,5.0,3.0,NaN,NaN,NaN,26.29,25.94,8002.61
3,6036,VMS 2,3,Abierta,50.0,6.0,NaN,NaN,NaN,NaN,55.45,52.44,2530.28
4,5981,VMS 2,3,Inseminada,111.0,5.0,1.0,NaN,NaN,NaN,47.71,44.75,4450.66


Se observa que existen varias categorias como VMS 1, VMS 2, VMS 3, ESTABLO INICIAL, MACHOS, BECERRAS/NOVILLAS, SECAS y RETO para clasificar el grupo en el que se encuentra el animal. Las vacas que se encuentran dentro de VMS 1, VMS 2 y VMS 3 contiene información sobre el estado de produccion y reproductivo de las vacas.

In [ ]:
global_hato_df['Nombre del grupo'].unique()

array(['VMS 1', 'VMS 2', 'VMS 3', 'ESTABLO INICIAL', 'MACHOS',
       'BECERRAS/NOVILLAS', 'SECAS', 'RETO'], dtype=object)

La categoria de ESTABLO INCIAL, MACHOS, BECCERRAS/NOVILLAS, SECAS y RETO se eliminan ya que no nos brindan información sobre vacas que se encuentran en produccion de leche.

In [ ]:
groups_to_remove = ['ESTABLO INICIAL', 'MACHOS', 'BECERRAS/NOVILLAS', 'SECAS', 'RETO']

global_hato_df = global_hato_df[~global_hato_df['Nombre del grupo'].isin(groups_to_remove)]

print("Valores únicos en 'Nombre del grupo' después de la eliminación:")
print(global_hato_df['Nombre del grupo'].unique())

Valores únicos en 'Nombre del grupo' después de la eliminación:
['VMS 1' 'VMS 2' 'VMS 3']


Las columnas seleecionadas contienen información reelevante para la predicción del modelo. Los features seleccionados son:
- Número del animal
- Estado de la reproducción
- Días en ordeño
- Nº Lactación
- Número de inseminaciones
- Días preñada
- Días para el parto
- Producción de leche ayer
- Producción media diaria últimos 7 días
- Producción TOTAL en lactación

In [ ]:
selected_columns = [
    'Número del animal',
    'Estado de la reproducción',
    'Nº Lactación',
    'Días en ordeño',
    'Número de inseminaciones',
    'Días preñada',
    'Días para el parto',
    'Producción de leche ayer',
    'Producción media diaria últimos 7 días',
    'Producción TOTAL en lactación'
    ]

global_hato_df = global_hato_df[selected_columns]

In [ ]:
display(global_hato_df)

,Número del animal,Estado de la reproducción,Nº Lactación,Días en ordeño,Número de inseminaciones,Días preñada,Días para el parto,Producción de leche ayer,Producción media diaria últimos 7 días,Producción TOTAL en lactación
0,5704,Inseminada,7.0,97.0,1.0,NaN,NaN,33.01,31.10,2675.94
1,5921,Abierta,6.0,65.0,NaN,NaN,NaN,33.70,37.79,2606.48
2,6003,Inseminada,5.0,219.0,3.0,NaN,NaN,26.29,25.94,8002.61
3,6036,Abierta,6.0,50.0,NaN,NaN,NaN,55.45,52.44,2530.28
4,5981,Inseminada,5.0,111.0,1.0,NaN,NaN,47.71,44.75,4450.66
...,...,...,...,...,...,...,...,...,...,...
531,1577,Preñada,3.0,374.0,4.0,194.0,89.0,29.73,27.35,16764.70
534,2107,Preñada,2.0,311.0,2.0,209.0,74.0,18.04,18.75,10927.03
547,2150,Preñada,2.0,306.0,1.0,228.0,55.0,25.38,25.94,11933.75
548,8715,Preñada,1.0,292.0,1.0,228.0,55.0,23.64,23.55,10061.69


Existe gran cantidad de datos nulos dentro del dataset, pero aún contiene información sobre la producción de leche.

In [ ]:
null_counts = global_hato_df.isnull().sum()
print("Número de valores nulos por columna:")
print(null_counts)

Número de valores nulos por columna:
Número del animal                           0
Estado de la reproducción                   0
Nº Lactación                                0
Días en ordeño                              0
Número de inseminaciones                   53
Días preñada                              121
Días para el parto                        121
Producción de leche ayer                    3
Producción media diaria últimos 7 días      1
Producción TOTAL en lactación               0
dtype: int64


Se sustituyen los valores nulos por cero para conservar la información relevante para el modelo.

In [ ]:
global_hato_df.fillna(0, inplace=True)

In [ ]:
null_counts = global_hato_df.isnull().sum()
print("Número de valores nulos por columna:")
print(null_counts)

Número de valores nulos por columna:
Número del animal                         0
Estado de la reproducción                 0
Nº Lactación                              0
Días en ordeño                            0
Número de inseminaciones                  0
Días preñada                              0
Días para el parto                        0
Producción de leche ayer                  0
Producción media diaria últimos 7 días    0
Producción TOTAL en lactación             0
dtype: int64


In [ ]:
display(global_hato_df)

,Número del animal,Estado de la reproducción,Nº Lactación,Días en ordeño,Número de inseminaciones,Días preñada,Días para el parto,Producción de leche ayer,Producción media diaria últimos 7 días,Producción TOTAL en lactación
0,5704,Inseminada,7.0,97.0,1.0,0.0,0.0,33.01,31.10,2675.94
1,5921,Abierta,6.0,65.0,0.0,0.0,0.0,33.70,37.79,2606.48
2,6003,Inseminada,5.0,219.0,3.0,0.0,0.0,26.29,25.94,8002.61
3,6036,Abierta,6.0,50.0,0.0,0.0,0.0,55.45,52.44,2530.28
4,5981,Inseminada,5.0,111.0,1.0,0.0,0.0,47.71,44.75,4450.66
...,...,...,...,...,...,...,...,...,...,...
531,1577,Preñada,3.0,374.0,4.0,194.0,89.0,29.73,27.35,16764.70
534,2107,Preñada,2.0,311.0,2.0,209.0,74.0,18.04,18.75,10927.03
547,2150,Preñada,2.0,306.0,1.0,228.0,55.0,25.38,25.94,11933.75
548,8715,Preñada,1.0,292.0,1.0,228.0,55.0,23.64,23.55,10061.69


In [ ]:
global_hato_df.to_csv('/content/drive/MyDrive/Vacas/global_hato_df.csv', index=False)